## Import các thư viện cần thiết

In [ ]:
import re #regex
import csv
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from pandas import DataFrame
import urllib.request
import joblib #load, dump pkl
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from underthesea import word_tokenize #word_tokenize
import numpy as np
import matplotlib.pyplot as plt
import transformers as ppb # load model BERT
from transformers import BertModel, BertTokenizer
import torch
from bs4 import BeautifulSoup

# 1. Data Visualization

## Load data

In [ ]:
file = 'tiki_comments.csv'
df = pd.read_csv(file,sep=",")

### Kích thước của bộ dữ liệu

Dữ liệu có 1399 dòng và 2 cột.

In [ ]:
df.shape

### Ý nghĩa các các cột

Kiểu dữ liệu của các cột

In [ ]:
dtypes = df.dtypes
dtypes

- Cột comment dạng chuỗi là bình luận của người dùng.
- Cột is_trust đánh giá bình luận là tích cực hoặc trung lập (1) hay tiêu cực (0). 
- Sử dụng cột is_trust làm label cho tập dữ liệu.

In [ ]:
pd.set_option('display.max_colwidth',1000)
df.head(10)

## Chuẩn hoá dữ liệu

### Xoá các dòng có giá trị cột comment trùng lặp.

In [ ]:
# Đếm số lượng dòng trùng lặp
num_duplicated_rows = df.duplicated(['comment']).sum()
num_duplicated_rows
# Xoá các dòng trùng
df.drop_duplicates(subset ="comment", keep = 'first', inplace = True) 

print(df.shape)
df.head(10)

### Chuẩn hoá comment
Dữ liệu hiện đang chứa những kí tự như dấu chấm câu, kí tự đặc biệt, icon,... Những kí tự này không cần thiết cho việc khai phá dữ liệu nên ta sẽ tiến hành loại bỏ chúng. Các bước loại bỏ:
- Xoá đi các dấu chấm câu như (. , ? ; ...)
- Xoá đi các kí tự đặc biệt: @,#,*,...
- Xoá khoảng trắng đầu và cuối mỗi comment, xoá  khoảng trắng dư thừa giữa các từ.
- Xoá đi icons và emoji.

In [ ]:
def standardize_data(row):
    row = re.sub(r"[\.,\?]+$-", "", row)
    row = re.sub(r'[^\w]',' ', row) 
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")
    row = row.strip()
    row = " ".join(row.split())
    return row

Comment trước khi chuẩn hoá

In [ ]:
df[20:30]

### Sau khi chuẩn hoá

In [ ]:
df['comment'] = df['comment'].apply(standardize_data)
df[20:30]

### Lưu dữ liệu phục vụ cho việc train model

In [ ]:
df.to_csv ('formated__dataset.csv', index = False, header=True)

## Trực quan hoá dữ liệu

<b>Câu hỏi:</b> Tỉ lệ các loại comment trong tập dữ liệu.<br>
Ta sẽ tính toán xem mỗi loại comment chiếm bao nhiêu phầm trăm (%) trong tập dữ liệu.

In [ ]:
#Tính % số lượng các loại comment
num_trust_cmt = df['is_trust'].value_counts()[1]
num_not_trust_cmt = df['is_trust'].value_counts()[0]

trust_ratio = ((num_trust_cmt / df.shape[0])*100).round(2);
not_trust_ratio = ((num_not_trust_cmt / df.shape[0])*100).round(2);
not_trust_ratio

data = [trust_ratio,not_trust_ratio]
labels = ['Tích cực', "Tiêu cực"]

fig = plt.figure(figsize =(10, 7)) 
plt.pie(data, labels = labels, explode = (0.1, 0), autopct='%1.1f%%',shadow=True) 
plt.title('Tỉ lệ % bình luận tích cực và tiêu cực')
plt.axis('equal')

plt.show() 
